# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR
#Relier les pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [8]:
#import the train dataset(1000 first rows)
df = pd.read_csv('../data/train.csv',  nrows=1000)

In [23]:
# prepare X and y
y = df.fare_amount
X = df.drop(columns=['fare_amount','key'])

In [24]:
X.shape

(1000, 6)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [33]:
X_train.head(2)

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
703,2012-05-21 11:37:42 UTC,-73.992170,40.764320,-73.966214,40.762562,1
311,2009-03-13 17:43:18 UTC,-73.977984,40.754546,-73.958947,40.779093,4


In [30]:
%%time
df = extract_time_features(df)

Wall time: 9 ms


In [54]:
df.shape

(1000, 12)

### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [83]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [105]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
                
        # A COMPPLETER
        self.start_lat= start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
    
    def fit(self, X, y=None):
        # A COMPLETER retourne l'objet lui meme 
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        #Calculer la distance
        X_[['distance']]= haversine_vectorized(X_)
        return X_[['distance']]

In [106]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
#transformer
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
703,2.194746
311,3.165517
722,2.430238
629,0.484442
0,1.030764


In [172]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.

    """

    def __init__(self, pickup_datetime="pickup_datetime"):

       # A COMPLETER
        self.pickup_datetime=pickup_datetime


    def fit(self, X, y=None):
        # A COMPLETER
        return self

    def transform(self, X, y=None):
        # A COMPLETER 
        X_ = X.copy()
        X_["pickup_datetime"] = pd.to_datetime(X_["pickup_datetime"]).dt.tz_convert("America/New_York")
        X_["dow"] = X_["pickup_datetime"].dt.dayofweek
        X_["hour"] = X_["pickup_datetime"].dt.hour
        X_["month"] = X_["pickup_datetime"].dt.month
        X_["year"] = X_["pickup_datetime"].dt.year
        return X_[['dow', 'hour', 'month', 'year']]

In [173]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
703,0,7,5,2012
311,4,13,3,2009
722,5,21,10,2010
629,1,12,8,2014
0,0,13,6,2009


###  Prepocessing pipeline

In [174]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [175]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures

In [176]:
# create distance pipeline dist_pipe
pipe_distance = Pipeline([
    ('DistanceTransformer', DistanceTransformer()),
    ('StandarScaler', StandardScaler()),
])

# display distance pipeline
pipe_distance

Pipeline(steps=[('DistanceTransformer', DistanceTransformer()),
                ('StandarScaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [177]:

# create time pipeline time_pipe
time_pipe =Pipeline([
    ('datetime', TimeFeaturesEncoder()),
    ('encoder', OneHotEncoder()),

])
# display time pipeline
time_pipe

Pipeline(steps=[('datetime', TimeFeaturesEncoder()),
                ('encoder', OneHotEncoder())])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [178]:
# create preprocessing pipeline preproc_pipe

preproc_pipe = ColumnTransformer(transformers=[('distance', pipe_distance,
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                  
                                ('time', time_pipe,
                                 ['pickup_datetime'])])
preproc_pipe                                

ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('DistanceTransformer',
                                                  DistanceTransformer()),
                                                 ('StandarScaler',
                                                  StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                ('time',
                                 Pipeline(steps=[('datetime',
                                                  TimeFeaturesEncoder()),
                                                 ('encoder', OneHotEncoder())]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [179]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
model = Pipeline([
    ('preproc_pipe', preproc_pipe),
    ('regression', LinearRegression())
])
# display the pipeline with model
model

Pipeline(steps=[('preproc_pipe',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('DistanceTransformer',
                                                                   DistanceTransformer()),
                                                                  ('StandarScaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('datetime',
                                                                   TimeFeaturesEncoder()),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['pickup_datetime'])])),
                ('regression', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [187]:
# train the pipelined model
train = model.fit(X_train, y_train)


# compute y_pred on the test set
y_pred = model.predict(X_test)


Use the RMSE to evaluate the performance of the model:

In [183]:
def compute_rmse(y_pred, y_true):
    rmse = np.sqrt(np.mean((y_pred-y_true)**2))
    return rmse

In [184]:
# call compute_rmse
compute_rmse(y_pred, y_test)

9.300762461057369

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [195]:
# implement get_data() function
def get_data(nrows=10000):
    '''returns a DataFrame with nrows'''
    df = pd.read_csv('../data/train.csv',  nrows=nrows)
    # A COMPLETER
    return df


In [185]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    # A COMPLETER
    df = df[df.fare_amount > 0]
    df = df[df.distance < 100]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count != 0]
    
    return df

In [188]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
     # A  COMPLETER
 
    pipe_distance = Pipeline([
        ('DistanceTransformer', DistanceTransformer()),
        ('StandarScaler', StandardScaler()),
    ])

    # create time pipeline time_pipe
    time_pipe =Pipeline([
        ('datetime', TimeFeaturesEncoder()),
        ('encoder', OneHotEncoder()),

    ])
    preproc_pipe = ColumnTransformer(transformers=[('distance', pipe_distance,
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                  
                                ('time', time_pipe,
                                 ['pickup_datetime'])])
    pipe = Pipeline([
    ('preproc_pipe', preproc_pipe),
    ('regression', LinearRegression())
    ])
    return pipe

In [197]:
# implement train() function
def train(X_train, y_train, pipeline_model):
    '''returns a trained pipelined model'''
    pipeline_model.fit(X_train, y_train)
 
   # A COMPLETER
    return pipeline_model

In [199]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    # A COMPLETER
    y_pred = pipeline.predict(X_test)
    rmse = np.sqrt(np.mean((y_pred-y_test)**2))
    #print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [205]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=32)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)
print(rmse)

9.793696700931271


In [30]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

9.396264153241109


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀